In [1]:
class ImageStandardizer:
    from PIL import Image
    import numpy as np    
    def __init__(self, max_thumb_size):
        self.max_thumb_size = max_thumb_size
    def array_from_file(self, filepath):

        with Image.open(filepath) as image:
            blank_image = Image.new('RGB', self.max_thumb_size, 'black')
            image.thumbnail(self.max_thumb_size, Image.BICUBIC)
            blank_image.paste(image, (0,0)) # add to left upper corner
            return np.asarray(blank_image)
    def array_from_file_list(self, filepaths):
        res=[]
        for filepath in filepaths:
            with Image.open(filepath) as image:
                blank_image = Image.new('RGB', self.max_thumb_size, 'black')
                image.thumbnail(self.max_thumb_size, Image.BICUBIC)
                blank_image.paste(image, (0,0)) # add to left upper corner
                res.append(np.asarray(blank_image))
        return np.asarray(res)
            
            
    def save_to_image_file(self, image, output_image_path):
        image.save(output_image_path)        

In [3]:
import numpy as np

In [60]:
sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)

In [4]:
from keras import models , optimizers , losses ,activations , callbacks
from keras.layers import *
import keras.backend as K
from keras import Sequential

Using TensorFlow backend.


In [8]:
mobileNetV2 = MobileNetV2(input_shape=(160,64,3),include_top=False)

In [9]:
mobileNetV2.get_output_at(0)

<tf.Tensor 'out_relu_1/Relu6:0' shape=(None, 5, 2, 1280) dtype=float32>

In [10]:
    basic_net = Sequential([mobileNetV2,Flatten(), Dense( 256 )])

In [11]:
basic_net.layers

In [12]:

        for layer in basic_net.layers[0].layers:
            layer.trainable = False
        
        input_x1 = Input( shape=(160,64,3))
        input_x2 = Input( shape=(160,64,3) )

        output_x1 = basic_net( input_x1 )
        output_x2 = basic_net( input_x2 )
  
   

In [13]:
      
        seq_similarity_model = [
            Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] )),
            #Lambda( lambda a, b : K.abs( a - b )),
            #Lambda( lambda tensors : K.abs( tensors)),
            Dense( 1 , activation=activations.sigmoid)
        ]
        similarity_model1 =  Sequential( seq_similarity_model )

In [14]:
similarity_model1.layers[0].input

AttributeError: Layer lambda_1 is not connected, no input to return.

In [16]:
[output_x1, output_x2]

[<tf.Tensor 'sequential_1/dense_1/BiasAdd:0' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'sequential_1_1/dense_1/BiasAdd:0' shape=(None, 256) dtype=float32>]

In [20]:
output_xs = K.stack([output_x1, output_x2], axis=1)

In [28]:
outputs = similarity_model1(inputs=output_xs)

In [40]:
Euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))
# use and add the distance function
Euc_distance = Euc_layer([output_x1, output_x2])
#identify the prediction
FC = Dense(1,activation='sigmoid')
prediction = FC(Euc_distance)
#Define the network with the left and right inputs and the ouput prediction
siamese_net = models.Model(inputs=[input_x1, input_x2],outputs=prediction)

In [36]:
FC.get_weights()[0].shape

(256, 1)

In [187]:
[output_x1, output_x2]

[<tf.Tensor 'sequential_20/dense_21/BiasAdd:0' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'sequential_20_1/dense_21/BiasAdd:0' shape=(None, 256) dtype=float32>]

In [39]:
input_xs = K.stack([input_x1, input_x2], axis=1)
model = models.Model( [input_x1, input_x2] , outputs )

AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [38]:
    #outputs = similarity_model1([output_x1, output_x2])

    model = models.Model( [ input_x1 , input_x2 ] , outputs )

AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [1]:
MAX_THUMB_SIZE=(32,80)

In [6]:
#from tensorflow.python.keras import models , optimizers , losses ,activations , callbacks
#from tensorflow.python.keras.layers import *
#import tensorflow.python.keras.backend as K
from keras import models , optimizers , losses ,activations , callbacks
from keras.layers import *
import keras.backend as K
from keras import Sequential
from keras.applications.mobilenet_v2 import MobileNetV2

from PIL import Image
#import tensorflow as tf
import time
import os
import numpy as np
import pickle

class Recognizer (object) :

    def __init__( self, max_thumb_size, model=None ):
        self.max_thumb_size = max_thumb_size
        self.input_shape = (self.max_thumb_size[1], self.max_thumb_size[0], 3)                 
        self.class_indices = {}
        
        if model:
            self._model = model
            #TODO: 
            return
        
        self._basic_model = self.basic_net_3()
        self._similarity_model = self.similarity_model()

        input_x1 = Input( shape=self.input_shape )
        input_x2 = Input( shape=self.input_shape )

        output_x1 = self._basic_model( input_x1 )
        output_x2 = self._basic_model( input_x2 )
        
        self.euc_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))
        euc_distance = self.euc_layer([output_x1, output_x2])
        self.finalFC = Dense(1, activation='sigmoid')
        prediction = self.finalFC(euc_distance)
        self._model = models.Model(inputs=[input_x1, input_x2],outputs=prediction)

        #it doesn't work when specifying optimizer through  class     self._model.compile( loss=losses.binary_crossentropy , optimizer=optimizers.Adam(lr=0.0001))
        adam = optimizers.Adam(lr=0.001)
        self._model.compile( loss=losses.binary_crossentropy , optimizer=adam, metrics=['accuracy'])
        #self._model.compile(optimizer='sgd', loss=losses.binary_crossentropy, metrics=['accuracy'])
        
    def basic_net(self):
        kernel_size_1 = ( 3 , 3 )
        kernel_size_2 = ( 3 , 3 )
        pool_size_1 = ( 3 , 3 )
        pool_size_2 = ( 2 , 2 )
        strides_1 = 2
        strides_2 = 1
        
        seq_conv_model = [
            Conv2D( 32, kernel_size=(11,11) , strides=2 , activation=activations.relu, input_shape=self.input_shape ),
            MaxPooling2D(),
            
            Conv2D( 64, kernel_size=(5,5) , strides=1 , activation=activations.relu ),
            MaxPooling2D(),
          
            #Conv2D( 256, kernel_size=(3,3) , strides=1 , activation=activations.relu ),
           
            Conv2D( 128, kernel_size=(3,3) , strides=1 , activation=activations.relu ),
            MaxPooling2D(),
         
            #Conv2D( 128, kernel_size=(3,3) , strides=1 , activation=activations.relu ),
            #MaxPooling2D(),
            Flatten(),
            Dense( 512 , activation=activations.relu ),
            Dense( 64 , activation=activations.relu ) 
        ]
        seq_model = Sequential( seq_conv_model )
        return seq_model
    
    def basic_net_2(self):
        kernel_size_1 = ( 3 , 3 )
        kernel_size_2 = ( 3 , 3 )
        pool_size_1 = ( 3 , 3 )
        pool_size_2 = ( 2 , 2 )
        strides_1 = 2
        strides_2 = 1
        
        seq_conv_model = [
            Conv2D( 8, kernel_size=kernel_size_1 , strides=strides_1 , activation=activations.relu, input_shape=self.input_shape ),
            MaxPooling2D(pool_size=pool_size_1, strides=strides_1 ),

            Conv2D( 16, kernel_size=kernel_size_2 , strides=strides_2 , activation=activations.relu ),
            MaxPooling2D(pool_size=pool_size_2 , strides=strides_2),

            Flatten(),
            Dense( 50 , activation=activations.sigmoid )
        ]
        seq_model = keras.Sequential( seq_conv_model )
        return seq_model
    def basic_net_3(self):        
        mobileNetV2 = MobileNetV2(input_shape=self.input_shape,include_top=False)
        for layer in mobileNetV2.layers:
            layer.trainable = False
        return Sequential([mobileNetV2, Flatten(), Dense( 64 )])

    def similarity_model(self):
        seq_similarity_model = [
            Lambda( lambda tensors : K.abs( tensors[0] - tensors[1] )),
            Dense( 1 , activation=activations.sigmoid)
        ]
        return Sequential( seq_similarity_model )

    def fit(self, X, Y ,   **kwargs  ):
        X1 = X[:,0,:,:,:]
        X2 = X[:,1,:,:,:]
        initial_time = time.time()
        self._model.fit( [X1,X2] , Y , **kwargs)                     
        final_time = time.time()
        
        eta = ( final_time - initial_time )
        time_unit = 'seconds'
        if eta >= 60 :
            eta = eta / 60
            time_unit = 'minutes'
        self._model.summary( )
        print( 'Elapsed time acquired -> {} {}'.format(  eta , time_unit ) )
        
    def fit_generator(self, train_generator ,  **kwargs  ):
        #TODO: to implement
        pass


    def calc_embeddings(self, dir_path, save_path='./embeddings.npy'):
        self.mapping_files=[]
        self.mapping_classes=[]
        arrays=[]
        sub_dirs = [f.name for f in os.scandir(dir_path) if f.is_dir()]
        for sd in sub_dirs:
            files =  [f.path for f in os.scandir(os.path.join(dir_path, sd)) if f.is_file()]
            if any(files):
                arrays.append( recognizer._basic_model.predict(ImageStandardizer(max_thumb_size).array_from_file_list(files)/255))
                mapping_files.extend(files)
                mapping_classes.extend([sd] * len(files))
        self.all_embeddings = np.vstack(tuple(arrays))
        if save_path:
            np.save(save_path, self.all_embeddings)
        
    def prepare_images_from_dir( self , dir_path , save_intermediate = True) :
        self.labels_dict={}
        self.labels_list=[]
        imageStandardizer = ImageStandardizer(self.max_thumb_size)  
        sub_dirs = [f.name for f in os.scandir(dir_path) if f.is_dir()]
        count=0
        arrays_per_label=[]
        for sd in sub_dirs:
            self.labels_dict[sd] = count
            self.labels_list.append(sd)
            
            files =  [f.path for f in os.scandir(os.path.join(dir_path, sd)) if f.is_file()]
            arrays_per_label.append(imageStandardizer.array_from_file_list(files)/255)
            count+=1   
        
        if save_intermediate:
            with open('arrays_per_label.pkl', 'wb') as file:
                pickle.dump(arrays_per_label, file)
        
        #TODO: add shuffling
        return arrays_per_label
    
    def prepare_pairs(self, arrays_per_label):
        x_genuine_pair=[]
        y_genuine=[]

        for label_arrays in arrays_per_label:
            len_arrays = len(label_arrays)
            for i in range(0, len_arrays-1):
                for j in range(i+1, len_arrays):
                    x_genuine_pair.append([label_arrays[i], label_arrays[j]])
                    y_genuine.append(1)
                

        x_imposite_pair=[]
        y_imposite=[]
                      
        for _ in range(self.imposite_pair_count):
            class_count = len(arrays_per_label)
            i, j = np.random.randint(0, class_count, 2)
            if i != j:
                if len(arrays_per_label[i]) > 0 and len(arrays_per_label[j]) > 0:
                    i1 = np.random.randint(0, len(arrays_per_label[i]))
                    j1 = np.random.randint(0, len(arrays_per_label[j]))

                    x_imposite_pair.append([arrays_per_label[i][i1], arrays_per_label[j][j1]])
                    y_imposite.append(0) 
                    

        X = np.concatenate([np.array(x_genuine_pair, dtype=np.float32), np.array(x_imposite_pair, dtype=np.float32)], axis=0)      
        Y = np.concatenate([np.array(y_genuine, dtype=np.float32), np.array(y_imposite, dtype=np.float32)], axis=0)        
                
        return X, Y
 
    def evaluate(self , test_X , test_Y  ) :
        X1 = test_X[:,0,:,:,:]
        X2 = test_X[:,1,:,:,:]
        return self._model.evaluate( [X1, X2] , test_Y)

    def predict(self, X  ):
        X1 = X[:,0,:,:,:]
        X2 = X[:,1,:,:,:]
        predictions = self._model.predict( [X1, X2])
        return predictions

    def summary(self):
        self._model.summary()

    def save_model(self , file_path ='model.h5'):
        self._model.save(file_path )

    def load_model(self , file_path ='model.h5'):
        self._model = models.load_model(file_path)
        self._basic_model = self._model.layers[2]
        self._similarity_model = self._model.layers[-1]
        
recognizer = Recognizer(MAX_THUMB_SIZE)

In [4]:
recognizer._model.layers[2].layers

In [7]:
recognizer.save_model('m3.h5')

In [6]:
DATA_DIR = r'd:\ML\data\f'
IMAGE_DIR  = os.path.join(DATA_DIR, 'images', "out")


In [77]:
del K
import tensorflow.python.keras.backend as K

In [5]:
K

<module 'tensorflow.python.keras.backend' from 'D:\\miniconda3\\lib\\site-packages\\tensorflow_core\\python\\keras\\backend.py'>

In [62]:
recognizer.load_model('m3_latest.h5')

In [49]:
        mapping_files=[]
        mapping_classes=[]
        arrays=[]
        dir_path = IMAGE_DIR
        sub_dirs = [f.name for f in os.scandir(dir_path) if f.is_dir()]
        for sd in sub_dirs:
            files =  [f.path for f in os.scandir(os.path.join(dir_path, sd)) if f.is_file()]
            if any(files):
                arrays.append( recognizer._basic_model.predict(ImageStandardizer(MAX_THUMB_SIZE).array_from_file_list(files)/255))

                mapping_files.extend(files)
                mapping_classes.extend([sd] * len(files))
        all_embeddings = np.vstack(tuple(arrays))
        all_embeddings.shape
            
            

(378, 64)

(378, 64)

In [55]:
from scipy.special import expit
test_image_path = r'D:\ML\data\f\images\out\CAM00003_118e394ea6e44235925aac3af3d7f3c7.jpg'
test_array = ImageStandardizer(MAX_THUMB_SIZE).array_from_file(test_image_path)/255
recognizing = recognizer._basic_model.predict(np.expand_dims(test_array, axis=0))

In [56]:
w, b =recognizer._similarity_model.layers[-1].get_weights()
x=np.abs(all_embeddings- recognizing)
scores = expit(np.dot(x,w)+b)
scores = scores[:,0]
scores.argsort()[-1:][::-1]

array([205], dtype=int64)

In [61]:
np.sum(np.abs(x), axis=1)

array([204.24582 , 179.14958 , 200.60437 , 210.33305 , 184.73126 ,
       197.70135 , 212.80118 , 235.13535 , 206.56067 , 223.77353 ,
       211.14041 , 226.8733  , 151.26743 , 172.78833 , 157.83691 ,
       164.68001 , 144.15228 , 136.5698  , 156.05261 , 164.69475 ,
       163.67218 , 138.77116 , 158.47293 , 151.85413 , 202.14146 ,
       186.9679  , 201.5093  , 215.00026 , 196.45094 , 205.56407 ,
       199.20813 , 211.24295 , 202.63396 , 208.54993 , 166.13788 ,
       186.24974 , 163.69366 , 172.71667 , 167.81148 , 155.15485 ,
       178.20052 , 171.74359 , 157.50061 , 257.0649  , 256.565   ,
       248.10886 , 240.22864 , 259.11108 , 249.32346 , 258.76135 ,
       248.12115 , 263.98993 , 252.29456 , 255.97473 , 258.52386 ,
       264.17255 , 251.90892 , 259.83313 , 258.64313 , 163.7786  ,
       191.94696 , 162.67422 , 160.01422 , 182.21606 , 161.9861  ,
       177.63016 , 177.15811 , 162.2586  , 144.3855  , 154.88121 ,
       191.81789 , 140.43857 , 193.04538 , 172.63387 , 167.062

In [64]:
scores

array([3.00779243e-06, 1.68552768e-04, 1.80405220e-06, 3.33428834e-05,
       1.58100590e-07, 2.28934582e-07, 3.43359581e-08, 2.48327797e-06,
       1.60541617e-07, 2.20813035e-06, 1.49112327e-06, 1.18685698e-07,
       7.54416704e-01, 5.27473427e-02, 1.95540246e-02, 1.47834967e-03,
       8.87076318e-01, 4.76872735e-03, 6.30598962e-01, 8.70609701e-01,
       8.60538721e-01, 2.92616606e-01, 8.06221783e-01, 9.61953521e-01,
       7.03636260e-07, 4.21358628e-08, 8.62632760e-07, 4.65848598e-06,
       1.04333807e-07, 8.13997936e-09, 1.36889096e-08, 2.33895619e-07,
       2.57386090e-09, 7.78955425e-08, 1.56258502e-05, 3.72761315e-06,
       1.02235390e-04, 2.13351564e-06, 1.00356147e-05, 6.46840226e-06,
       6.14345936e-06, 1.93810661e-06, 1.77935453e-05, 9.17895013e-12,
       1.88015272e-12, 2.98724309e-11, 1.76629822e-11, 1.60555629e-12,
       4.30642431e-11, 2.77153842e-13, 7.67419114e-11, 4.43633975e-12,
       1.87042374e-12, 4.81350723e-12, 7.99952032e-12, 2.07969883e-12,
      

In [57]:
indices = np.where(scores>0.5)[0]
indices

array([  2,  12,  14,  16,  18,  19,  21,  22,  50,  52,  55,  57,  60,
        63,  76,  79, 107, 108, 109, 110, 111, 112, 113, 114, 117, 119,
       155, 159, 160, 161, 163, 164, 165, 166, 168, 170, 171, 175, 184,
       185, 190, 198, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215,
       217, 218, 219, 220, 221, 224, 247, 268, 269, 270, 274, 297, 302,
       303, 317, 322, 336, 337, 339, 345, 346, 351, 353, 355, 356, 358,
       362], dtype=int64)

In [62]:
K=15
indices = np.argpartition(scores, -K)[-K:]

In [551]:
np.max(expit(np.dot(x,w)+b))

0.99999666

In [16]:
mapping_classes[74]

'Василий Салосятов _16'

In [58]:
class_scores = [(mapping_classes[i], scores[i]) for i in indices]
class_scores
most_possible = {}
for c, s in class_scores:
    if c in most_possible:
        most_possible[c].append(s)
    else:
        most_possible[c] = [s]


In [59]:
for k, v in most_possible.items():
    print(k, np.mean(v), np.std(v))

Акоп _36 0.5669395 0.0
Александр Кулявцев _12 0.6037039 0.06522206
Араз Эйвазов _13 0.59534603 0.031525046
Валера _44 0.58407426 0.06576496
Виталий Притулин _10 0.60051274 0.061988097
Владимир Федунов _9 0.61844015 0.07833812
Денис Глобчанский _3 0.66949815 0.10564791
Дмитрий Витвицкий _21 0.56239855 0.011429191
Дмитрий Морозов _18 0.6012496 0.09586704
Дмитрий Пятницкий _8 0.64786965 0.026243567
Иван Кошелапов _19 0.6917017 0.087107524
Игорь Дёмин _32 0.67347246 0.09362993
Мага _41 0.5066209 0.0
Максим Камышик _29 0.5462571 0.031700887
Михаил Невинный _38 0.6327678 0.0
Павел _43 0.5463175 0.0
Сандро Никурадзе _7 0.671219 0.029287905
Сергей _42 0.6384949 0.08891553
Тигран _6 0.6102081 0.06587339


In [68]:
class_scores

[('Александр Кулявцев _12', 0.7544167),
 ('Александр Кулявцев _12', 0.8870763),
 ('Александр Кулявцев _12', 0.63059896),
 ('Александр Кулявцев _12', 0.8706097),
 ('Александр Кулявцев _12', 0.8605387),
 ('Александр Кулявцев _12', 0.8062218),
 ('Александр Кулявцев _12', 0.9619535),
 ('Владимир Федунов _9', 0.52586055),
 ('Владимир Федунов _9', 0.9568923),
 ('Владимир Федунов _9', 0.9981055),
 ('Владимир Федунов _9', 0.69691235),
 ('Владимир Федунов _9', 0.6129267),
 ('Владимир Федунов _9', 0.66939205),
 ('Владимир Федунов _9', 0.97694254),
 ('Владимир Федунов _9', 0.99544),
 ('Владимир Федунов _9', 0.67002517),
 ('Владимир Федунов _9', 0.9721895),
 ('Владимир Федунов _9', 0.98308325),
 ('Владимир Федунов _9', 0.71765125),
 ('Владимир Федунов _9', 0.88908863),
 ('Владимир Федунов _9', 0.8861709),
 ('Владимир Федунов _9', 0.8838218),
 ('Владимир Федунов _9', 0.5410151),
 ('Владимир Федунов _9', 0.9356848),
 ('Владимир Федунов _9', 0.6587291),
 ('Владимир Федунов _9', 0.9180174),
 ('Владими

In [573]:
sorted(most_possible.items(), key=lambda item: item[1], reverse=True)[:3]

[('Александр Порожкин _14', 1.0),
 ('Араз Эйвазов _13', 1.0),
 ('Валера _44', 1.0)]

In [508]:
from itertools import groupby

In [554]:
[(k, (np.array(list(l))[:,1])) for k, l  in groupby(class_scores, lambda x:x[0])]

[('Виталий Притулин _10', array(['0.8518551', '0.83667743'], dtype='<U20')),
 ('Илья _22', array(['0.82544804', '0.77438444'], dtype='<U10')),
 ('Шахин Абулов _4',
  array(['0.96198285', '0.99813855', '0.9960905', '0.99992895',
         '0.99999666', '0.9718878', '0.96991897'], dtype='<U15'))]

In [514]:
groupby(class_scores, lambda x:x[0])

In [505]:
labels_list = np.unique(mapping_classes)
labels_list

array(['Акоп _36', 'Александр _11', 'Александр Кулявцев _12',
       'Александр Порожкин _14', 'Араз Эйвазов _13', 'Валера _44',
       'Василий Салосятов _16', 'Виталий Притулин _10',
       'Владимир Федунов _9', 'Денис Глобчанский _3',
       'Дмитрий Витвицкий _21', 'Дмитрий Морозов _18',
       'Дмитрий Пятницкий _8', 'Иван Кошелапов _19', 'Илья _22',
       'Максим Аникеев _33', 'Максим Камышик _29', 'Михаил Невинный _38',
       'Николай Новик _2', 'Павел _43', 'Сандро Никурадзе _7',
       'Сергей Стрельцов _23', 'Тигран _6', 'Шахин Абулов _4'],
      dtype='<U22')

In [555]:
np.array(mapping_files)[indices]

array(['d:\\ML\\data\\f\\images\\out\\Виталий Притулин _10\\VID_20200217_185954_ecf52e.jpg',
       'd:\\ML\\data\\f\\images\\out\\Виталий Притулин _10\\VID_20200217_193008_714329.jpg',
       'd:\\ML\\data\\f\\images\\out\\Илья _22\\CAM00003_0379ce4be376405083f8f96a7fdda46e.jpg',
       'd:\\ML\\data\\f\\images\\out\\Илья _22\\CAM00003_5d19fa94febd48a4a8ba693339ca7c36.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_003f646b01d144b095a8d19f641dcf02.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_2d227a9b5f4347edb08dea48133c605b.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_2f90de8ae52b47db88f9b3e3b198077c.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_34b1e79b23b54577a15fabbbe2a85f4f.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_76f5f6e12b454dba8eb8417e5eeecc59.jpg',
       'd:\\ML\\data\\f\\images\\out\\Шахин Абулов _4\\CAM00003_7d230ee7d3af4e2d9c3a33dec8861e32.jpg',
    

In [494]:
scores[indices]

array([0.9908873 , 0.9928229 , 0.9929639 , 0.9947424 , 0.99476033,
       0.9950682 , 0.99613404, 0.99529904, 0.99680245, 0.99996305,
       0.9968208 , 0.99955434, 0.9991129 , 0.9987306 , 0.9995023 ],
      dtype=float32)

In [495]:
(values,counts) = np.unique(np.array(mapping_classes)[indices],return_counts=True)
(values,counts)

(array(['Александр _11', 'Владимир Федунов _9', 'Дмитрий Витвицкий _21',
        'Дмитрий Пятницкий _8', 'Иван Кошелапов _19', 'Илья _22'],
       dtype='<U22'), array([1, 4, 4, 1, 2, 3], dtype=int64))

(array(['Илья _22'], dtype='<U22'), array([3], dtype=int64))

In [412]:
mapping_classes[index]

'Максим Аникеев _33'

In [ ]:
test_image = 

In [368]:

x.shape

(246, 64)

In [364]:
all_embeddings[2]

array([ 0.      ,  0.      ,  9.895573,  0.      ,  0.      , 30.561514,
       27.17593 , 20.026007,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      , 32.7002  ,  0.      ,  0.      , 31.940073, 14.668335,
        0.      , 20.192516,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      , 42.856277,  0.      , 29.251688, 16.613678,  0.      ,
        0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
       13.600231,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
       29.68519 ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
        0.      ,  0.      ,  0.      , 34.80243 , 10.774771,  0.      ,
       10.204382,  0.      ,  0.      ,  0.      , 18.60993 , 18.822977,
        0.      ,  0.      ,  0.      ,  0.      ], dtype=float32)

In [375]:
w.T.shape, b.shape

((1, 64), (1,))

In [384]:
(expit(np.dot(x,w)+b))[11:21]

array([[1.5538806e-12],
       [8.7822236e-11],
       [4.5586152e-12],
       [5.6297963e-11],
       [5.1387922e-11],
       [7.9003996e-01],
       [5.4325867e-01],
       [9.9967325e-01],
       [9.9908948e-01],
       [5.9550645e-20]], dtype=float32)

In [306]:
class_indices=26 #TODO: remove
embeddings = [[]]*class_indices
for i in range(class_indices):
    for j in range(len(arrays_per_label[i])):
        embeddings[i].append(recognizer._basic_model.predict(np.expand_dims(arrays_per_label[i][j], axis=0)))

array([[0.54326606, 0.54326606]], dtype=float32)

In [291]:
embeddings[i][j].T.shape

(64, 1)

In [308]:
all_embeddings = np.vstack([embeddings[i][j], embeddings[i][j]])

In [385]:
mapping_files[11:21]

['d:\\ML\\data\\f\\images\\out\\Александр _11\\CAM00003_8fff131ae2bd4c519c38594bfa2ccb0b.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр _11\\CAM00003_9147112a1f2f407b9a0a8de09b55ebeb.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр _11\\CAM00003_a475afbb161f471d8ae5ddc39b6faa66.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр _11\\CAM00003_c181d5f6a0ba4051bfb5ce6aad08b736.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр _11\\CAM00003_ea20766d519a4d12b0c8a484b8773511.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр Кулявцев _12\\CAM00003_15ed234ed5f143d38915126022fbf9df.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр Кулявцев _12\\CAM00003_259aa7a0af4d4b89919873d8ef13991a.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр Кулявцев _12\\CAM00003_f5af2765ba254948ade4cbe6082778f4.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр Кулявцев _12\\CAM00003_fcf13044493643bb9b70ba967dd316f5.jpg',
 'd:\\ML\\data\\f\\images\\out\\Александр Порожкин _14\\CAM00003_3af90fb8312948fe8cb4c84980d8e13c.jpg']

In [262]:
recognizer._similarity_model.layers[0].input

[<tf.Tensor 'sequential_20/Identity:0' shape=(None, 64) dtype=float32>,
 <tf.Tensor 'sequential_20_1/Identity:0' shape=(None, 64) dtype=float32>]

In [264]:
embeddings[i][j].shape

(1, 64)

In [481]:
arrays_per_label = recognizer.prepare_images_from_dir(path)
X, Y = recognizer.prepare_pairs(arrays_per_label)

In [480]:
path

'd:\\ML\\data\\f\\images\\out'

In [482]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [563]:
parameters = {
    'batch_size' : 32 ,
    'epochs' : 20 ,
    'callbacks' : None , # [ TensorBoard( log_dir='logs/{}'.format( time.time() ) ) ] ,
    'validation_data' : None
}

In [197]:
x_train[:, 0, :,:,:].shape

(6030, 150, 60, 3)

In [483]:
X1 = x_train[:,0,:,:,:]
X2 = x_train[:,1,:,:,:]

In [214]:
recognizer._model.fit([X1, X2], y_train)

KeyboardInterrupt: 

In [229]:
recognizer._similarity_model.layers[-1].get_weights()

[array([[ 0.12452415],
        [ 0.09079502],
        [ 0.03064282],
        [ 0.07271424],
        [-0.05793344],
        [-0.2663393 ],
        [-0.14829277],
        [-0.28353915],
        [-0.18515424],
        [-0.26085302],
        [ 0.14630626],
        [ 0.27739662],
        [-0.18752863],
        [-0.17392169],
        [-0.15702446],
        [-0.09803367],
        [-0.07001007],
        [-0.02977801],
        [-0.0023493 ],
        [-0.13726158],
        [-0.05140093],
        [-0.01243227],
        [ 0.19566163],
        [ 0.17835909],
        [-0.00054232],
        [-0.26771128],
        [ 0.1869823 ],
        [-0.0862117 ],
        [ 0.08208051],
        [ 0.19245052],
        [ 0.26505953],
        [ 0.11447033],
        [-0.25132498],
        [ 0.24162523],
        [ 0.04448824],
        [ 0.116676  ],
        [ 0.16571474],
        [ 0.2923946 ],
        [ 0.07219257],
        [-0.05919583],
        [ 0.27779028],
        [ 0.2653152 ],
        [ 0.11637916],
        [ 0

In [566]:
recognizer.fit(x_train, y_train, **parameters)

Train on 5478 samples
Epoch 1/20
5478/5478 [==============================] - 72s 13ms/sample - loss: 2.5698e-04
Epoch 2/20
5478/5478 [==============================] - 67s 12ms/sample - loss: 1.2297e-04
Epoch 3/20
5478/5478 [==============================] - 64s 12ms/sample - loss: 9.2845e-05
Epoch 4/20
5478/5478 [==============================] - 65s 12ms/sample - loss: 7.5423e-05
Epoch 5/20
5478/5478 [==============================] - 63s 11ms/sample - loss: 6.3373e-05
Epoch 6/20
5478/5478 [==============================] - 63s 11ms/sample - loss: 5.4226e-05
Epoch 7/20
5478/5478 [==============================] - 64s 12ms/sample - loss: 4.7013e-05
Epoch 8/20
5478/5478 [==============================] - 64s 12ms/sample - loss: 4.1204e-05
Epoch 9/20
5478/5478 [==============================] - 64s 12ms/sample - loss: 3.6336e-05
Epoch 10/20
5478/5478 [==============================] - 63s 11ms/sample - loss: 3.2218e-05
Epoch 11/20
5478/5478 [==============================] - 63s 11ms/s

In [565]:
import gc
gc.collect()

51156

In [486]:
recognizer.save_model('model.h5')

In [488]:
x_test.shape

(1826, 2, 150, 60, 3)

In [567]:
recognizer.evaluate(x_test, y_test)

1826/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.02549400829453764

In [50]:
max_thumb_size
inversed_max_thumb_size=(max_thumb_size[1], max_thumb_size[0])

In [58]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(rotation_range=10, rescale=1./255, fill_mode='constant')

gen = aug.flow_from_directory(path, save_to_dir = r'd:\ML\data\f\images\out5', target_size= inversed_max_thumb_size)
a = next(gen)
a.shape

Found 285 images belonging to 26 classes.


AttributeError: 'tuple' object has no attribute 'shape'

In [113]:
gen


In [88]:
a= next(gen)
a[0].shape, a[1].shape

(32, 150, 60, 3)

In [90]:
b= next(gen)
b[0].shape, b[1].shape

((32, 150, 60, 3), (32, 26))

In [103]:
batch_size = len(a[1])
y=np.zeros((batch_size,))                
for i in range (batch_size):
    y[i] = np.array_equal(a[1][i], b[1][i])
y                
    

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [108]:
def prepared_generator(gen):
    while True:
        a= next(gen)
        b= next(gen)
        batch_size = len(a[1])
        y=np.zeros((batch_size,))                
        for i in range (batch_size):
            y[i] = np.array_equal(a[1][i], b[1][i])

        yield ([a[0], b[0]], y)              


In [146]:
import keras
class PairGenerator(keras.preprocessing.image.DirectoryIterator):
    def __init__(self, gen):
        super().__init__(gen.directory, gen.image_data_generator)
        self.gen = gen
    def __next__(self):
        print('!')
        while True:
            a= next(self.gen)
            b= next(self.gen)
            batch_size = self.batch_size
            y=np.zeros((len(a[1]),))                
            for i in range (batch_size):
                y[i] = np.array_equal(a[1][i], b[1][i])

            yield [a[0], b[0]], y             

pair_gen = PairGenerator(gen)       
next(next(pair_gen))        

Found 285 images belonging to 26 classes.
!


([array([[[[0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           ...,
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ]],
  
          [[0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           ...,
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ]],
  
          [[0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           ...,
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ],
           [0.        , 0.        , 0.        ]],
  
          ...,
  
          [[0.        , 0.        , 0.  

In [110]:
gener = iter(prepared_generator)
next(prepared_generator)

TypeError: 'function' object is not iterable